In [ ]:
def get_descriptor_model(shape):
  
    model = VGG16(include_top=False, input_shape=(224,224,3), weights='imagenet')

    # Freeze all the layers
    for layer in model.layers[:]:
        layer.trainable = False

    init_weights = keras.initializers.he_normal()

    output = model.get_layer('block5_conv3').output

    output = Conv2D(64, 3, padding = 'valid', use_bias = True, kernel_initializer=init_weights)(output)
    output = BatchNormalization(axis = -1)(output)
    output = Activation('relu')(output)
    
    output = Conv2D(128, 3, padding = 'valid', use_bias = True, kernel_initializer=init_weights)(output)
    output = BatchNormalization(axis = -1)(output)
    output = Activation('relu')(output)
    
    output = Conv2D(128,3, padding = 'valid', use_bias = True, kernel_initializer=init_weights)(output)
    output = BatchNormalization(axis = -1)(output)
    output = Activation('relu')(output)
    output = Dropout(0.3)(output)
    
    output = Conv2D(128, 8, padding = 'valid', use_bias = True, kernel_initializer=init_weights)(output)

    output = Reshape((128,))(output)
    
    model = Model(model.input, output)
    
    newInput = Input(shape)
    i_concat = Concatenate()([newInput, newInput, newInput])
    i_resize = Lambda(lambda image: ktf.image.resize_images(image, (224,224)))(i_concat)
    newOutput = model(i_resize)
    model = Model(newInput, newOutput)
    return model